Reference: https://github.com/alineberry/my-movie-recommender/blob/master/notebooks/movie_similarity/autoencoder.ipynb

# Models: Movie Genres Sparse Autoencoder

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import numpy as np
import pandas as pd
from bunch import Bunch

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam

import pytorch_common.util as pu
from pytorch_common.modules.fn import Fn
from pytorch_common.callbacks import SaveBestModel
from pytorch_common.callbacks.output import Logger

from pytorch_common.util import set_device_name, \
                                get_device, \
                                LoggerBuilder

import model as ml
import data as dt
import data.dataset as ds

import data.plot as pl
import data as dtjo

import logging
import random

import recommender as rc

<Figure size 432x288 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
cpu = torch.device("cpu")
gpu = pu.get_device()

In [7]:
torch.cuda.is_available()

True

In [8]:
torch.__version__

'1.11.0'

In [9]:
def set_seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)

In [10]:
set_seed(42)

In [11]:
FIELD = 'genres'
WEIGHTS_PATH   = f'../weights/{FIELD}-tf-idf-sparse-auto-encoder.pt'
EMBEDDING_PATH = f'../datasets/movie_{FIELD}_embedding.json'

## Carga de dataset

In [12]:
def to_tensor(obs, device, columns): 
    data = obs[columns]
    if type(data) == pd.DataFrame:
        data = data.values
    return torch.tensor(data).to(device)

transform_fn = lambda obs, device: to_tensor(obs, device, [f'movie_{FIELD}'])

dataset = ds.MovieLensTMDBDatasetFactory.from_path(
    transform        = transform_fn,
    target_transform = transform_fn,
    device           = cpu,
    filter_fn        = lambda df: df[(df['user_movie_rating_year'] >= 1990) & (df['user_movie_rating_year'] <= 2019)]
)
dataset.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199791 entries, 0 to 199790
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   user_id                      199791 non-null  int64         
 1   user_seq                     199791 non-null  int64         
 2   user_movie_tags              199791 non-null  object        
 3   user_movie_rating            199791 non-null  int64         
 4   user_movie_rating_timestamp  199791 non-null  datetime64[ns]
 5   user_movie_rating_year       199791 non-null  int64         
 6   movie_id                     199791 non-null  int64         
 7   movie_seq                    199791 non-null  int64         
 8   movie_title                  199791 non-null  string        
 9   movie_genres                 199791 non-null  object        
 10  movie_for_adults             199791 non-null  bool          
 11  movie_original_language   

Select movies overview and add new curated tokens column:

In [13]:
columns = ['movie_id', 'movie_title', f'movie_{FIELD}']

movie_data = dataset \
    .data \
    .pipe(dt.select, columns) \
    .pipe(dt.distinct, ['movie_id']) \
    .pipe(dt.rename, {
        'movie_id': 'id', 
        'movie_title': 'title', 
        f'movie_{FIELD}': FIELD
    }) \
    .pipe(dt.join_str_list, FIELD) \
    .pipe(dt.tokenize, FIELD) \
    .pipe(dt.reset_index)

movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18703 entries, 0 to 18702
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             18703 non-null  int64 
 1   title          18703 non-null  string
 2   genres         18703 non-null  object
 3   genres_tokens  18703 non-null  object
dtypes: int64(1), object(2), string(1)
memory usage: 584.6+ KB


In [14]:
tfidf = movie_data.pipe(dt.tf_idf, f'{FIELD}_tokens')

tfidf.shape

(18703, 23)

## Definicion del modelo

In [15]:
def train(auto_encoder, tfidf, params):
    train_set = DataLoader(
        ds.TfIdfDataset(tfidf), 
        params.batch_size, 
        num_workers=params.n_workers, 
        pin_memory=True,
        shuffle=True
    )
    ml.AutoEncoderTrainer(auto_encoder).fit(
        train_set,
        loss_fn = ml.MSELossFn(reduction='elementwise_mean'),
        epochs  = params.epochs,
        encoder_optimizer = Adam(auto_encoder.encoder.parameters(), lr= params.lr),
        decoder_optimizer = Adam(auto_encoder.decoder.parameters(), lr= params.lr),
        callbacks=[Logger(['time', 'epoch', 'train_loss'])]
    )

## Entrenamiento

In [16]:
params = Bunch({
    'lr': 0.01,
    'epochs': 20,
    'n_workers': 24,
    'batch_size': 128,
    'sequence_size':  tfidf.shape[1],
    'intermediate_size': 5000,
    'encoded_size': 5,
    'experiment_name': f'{FIELD}-tf-idf-sparse-auto-encoder',
    'device': get_device()
})

In [17]:
auto_encoder = ml.AutoEncoder(
    params.sequence_size, 
    params.intermediate_size, 
    params.encoded_size
).to(get_device())
print(auto_encoder)

AutoEncoder(
  (encoder): Encoder(
    (mlp): Sequential(
      (0): Linear(in_features=23, out_features=5000, bias=True)
      (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=5000, out_features=5, bias=True)
      (5): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): Linear(in_features=5, out_features=5000, bias=True)
      (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=5000, out_features=23, bias=True)
      (5): BatchNorm1d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Sigmoid()
    )
  )
)

In [28]:
train(auto_encoder, tfidf, params)

2022-06-07 00:03:30,756 - INFO - {'time': '0:00:03.73', 'epoch': 1, 'train_loss': 0.010538066889406467}
2022-06-07 00:03:34,448 - INFO - {'time': '0:00:03.69', 'epoch': 2, 'train_loss': 0.010360929430747519}
2022-06-07 00:03:38,134 - INFO - {'time': '0:00:03.69', 'epoch': 3, 'train_loss': 0.010168045575470746}
2022-06-07 00:03:41,817 - INFO - {'time': '0:00:03.68', 'epoch': 4, 'train_loss': 0.010192163763757872}
2022-06-07 00:03:45,586 - INFO - {'time': '0:00:03.77', 'epoch': 5, 'train_loss': 0.010047743914230746}
2022-06-07 00:03:49,292 - INFO - {'time': '0:00:03.71', 'epoch': 6, 'train_loss': 0.009851457852059278}
2022-06-07 00:03:53,007 - INFO - {'time': '0:00:03.71', 'epoch': 7, 'train_loss': 0.00948029756862797}
2022-06-07 00:03:56,832 - INFO - {'time': '0:00:03.82', 'epoch': 8, 'train_loss': 0.009339952752703712}
2022-06-07 00:04:00,617 - INFO - {'time': '0:00:03.78', 'epoch': 9, 'train_loss': 0.00923601058343438}
2022-06-07 00:04:04,361 - INFO - {'time': '0:00:03.74', 'epoch': 1

In [29]:
torch.save(auto_encoder.state_dict(), WEIGHTS_PATH)

## Generacion de embeddings

In [30]:
embedding = auto_encoder.to(cpu).encode_from_batch(torch.tensor(tfidf.toarray()))
embedding.shape

torch.Size([18703, 5])

In [31]:
movie_data = movie_data \
    .pipe(dt.append_emb_vectors, embedding, FIELD) \
    .pipe(dt.drop, [f'{FIELD}_tokens'])

movie_data.to_json(EMBEDDING_PATH)
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18703 entries, 0 to 18702
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                18703 non-null  int64 
 1   title             18703 non-null  string
 2   genres            18703 non-null  object
 3   genres_embedding  18703 non-null  object
dtypes: int64(1), object(2), string(1)
memory usage: 584.6+ KB


## Evaluación

In [32]:
recommender = rc.DistanceMatrixRecommender(
    df     = pd.read_json(EMBEDDING_PATH), 
    column = f'{FIELD}_embedding', 
    device = get_device()
)

Building Distances Matrix:   0%|          | 0/18703 [00:00<?, ?it/s]

In [33]:
recommender.recommend(item_index=0).show()


Recommender: genres
Item


,id,title
0,1,Toy Story


Recommendations


,distance,id,title,genres
0,0,1,Toy Story,Adventure Animation Children Comedy Fantasy
2511,0,166461,Moana,Adventure Animation Children Comedy Fantasy
2266,0,114552,"Boxtrolls, The",Adventure Animation Children Comedy Fantasy
2821,0,4016,"Emperor's New Groove, The",Adventure Animation Children Comedy Fantasy
1916,0,53121,Shrek the Third,Adventure Animation Children Comedy Fantasy
